# CSC582: Relational to Key-Value Database Migration
## E-Commerce Database → Redis Cluster
**Mapping Format:** `TableName:TupleID:Attribute` → Value

**Cluster:** 4 Masters + 4 Replicas

## Step 1: Install Redis

In [23]:
!apt-get update -qq
!apt-get install -y redis-server redis-tools > /dev/null
!pip install redis -q
print('✅ Redis installed!')

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Redis installed!


## Step 2: Start 8 Redis Nodes (4 Masters + 4 Replicas)

In [24]:
import subprocess, time, os
for port in range(7000, 7008):
    os.makedirs(f'/tmp/redis-{port}', exist_ok=True)
    subprocess.run(['redis-server', '--port', str(port), '--daemonize', 'yes', '--dir', f'/tmp/redis-{port}'])
    print(f'Started node on port {port}')
time.sleep(2)
print('\n✅ All 8 nodes started!')

Started node on port 7000
Started node on port 7001
Started node on port 7002
Started node on port 7003
Started node on port 7004
Started node on port 7005
Started node on port 7006
Started node on port 7007

✅ All 8 nodes started!


In [25]:
# Configure Replication
for replica, master in [(7004,7000), (7005,7001), (7006,7002), (7007,7003)]:
    subprocess.run(['redis-cli', '-p', str(replica), 'REPLICAOF', 'localhost', str(master)])
    print(f'Replica {replica} → Master {master}')
time.sleep(1)
print('\n✅ Replication configured!')

Replica 7004 → Master 7000
Replica 7005 → Master 7001
Replica 7006 → Master 7002
Replica 7007 → Master 7003

✅ Replication configured!


In [26]:
import redis
masters = {i: redis.Redis(port=7000+i, decode_responses=True) for i in range(4)}
replicas = {i: redis.Redis(port=7004+i, decode_responses=True) for i in range(4)}

def get_slot(key):
    crc = 0
    for c in key.encode(): crc = ((crc << 5) + crc + c) & 0xFFFF
    return crc % 16384

def get_master_id(key):
    slot = get_slot(key)
    return slot // 4096

print('✅ Connected to all nodes!')

✅ Connected to all nodes!


## Part 1: Relational Database

In [27]:
import sqlite3, pandas as pd
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.executescript('''
CREATE TABLE Customer (customer_id INT PRIMARY KEY, first_name TEXT, last_name TEXT, email TEXT, phone TEXT, city TEXT, country TEXT);
CREATE TABLE Product (product_id INT PRIMARY KEY, product_name TEXT, category TEXT, price REAL, stock_quantity INT);
CREATE TABLE Order_ (order_id INT PRIMARY KEY, customer_id INT, order_date TEXT, status TEXT, total_amount REAL);
''')
cursor.executemany('INSERT INTO Customer VALUES (?,?,?,?,?,?,?)', [(1,'Ahmed','Al-Rashid','ahmed.rashid@email.com','+966501234567','Riyadh','Saudi Arabia'),(2,'Fatima','Hassan','fatima.hassan@email.com','+966502345678','Jeddah','Saudi Arabia'),(3,'Mohammed','Al-Saud','mohammed.saud@email.com','+966503456789','Dammam','Saudi Arabia'),(4,'Sara','Abdullah','sara.abdullah@email.com','+966504567890','Riyadh','Saudi Arabia'),(5,'Khalid','Omar','khalid.omar@email.com','+966505678901','Mecca','Saudi Arabia')])
cursor.executemany('INSERT INTO Product VALUES (?,?,?,?,?)', [(101,'Laptop Pro 15','Electronics',4500,25),(102,'Wireless Mouse','Electronics',150,100),(103,'Office Chair','Furniture',850,30),(104,'Standing Desk','Furniture',2200,15),(105,'Headphones','Electronics',1200,50),(106,'USB-C Hub','Electronics',280,75)])
cursor.executemany('INSERT INTO Order_ VALUES (?,?,?,?,?)', [(1001,1,'2024-06-01','delivered',4650),(1002,2,'2024-06-05','delivered',3050),(1003,1,'2024-06-10','shipped',1200),(1004,3,'2024-06-15','processing',2480),(1005,4,'2024-06-20','pending',850),(1006,5,'2024-06-25','delivered',4780),(1007,2,'2024-07-01','shipped',430)])
conn.commit()
print('✅ Database created!')

✅ Database created!


In [28]:
print('👥 CUSTOMER TABLE')
display(pd.read_sql_query('SELECT * FROM Customer', conn))

👥 CUSTOMER TABLE


,customer_id,first_name,last_name,email,phone,city,country
0,1,Ahmed,Al-Rashid,ahmed.rashid@email.com,+966501234567,Riyadh,Saudi Arabia
1,2,Fatima,Hassan,fatima.hassan@email.com,+966502345678,Jeddah,Saudi Arabia
2,3,Mohammed,Al-Saud,mohammed.saud@email.com,+966503456789,Dammam,Saudi Arabia
3,4,Sara,Abdullah,sara.abdullah@email.com,+966504567890,Riyadh,Saudi Arabia
4,5,Khalid,Omar,khalid.omar@email.com,+966505678901,Mecca,Saudi Arabia


In [29]:
print('📦 PRODUCT TABLE')
display(pd.read_sql_query('SELECT * FROM Product', conn))

📦 PRODUCT TABLE


,product_id,product_name,category,price,stock_quantity
0,101,Laptop Pro 15,Electronics,4500.0,25
1,102,Wireless Mouse,Electronics,150.0,100
2,103,Office Chair,Furniture,850.0,30
3,104,Standing Desk,Furniture,2200.0,15
4,105,Headphones,Electronics,1200.0,50
5,106,USB-C Hub,Electronics,280.0,75


In [30]:
print('🛒 ORDER TABLE')
display(pd.read_sql_query('SELECT * FROM Order_', conn))

🛒 ORDER TABLE


,order_id,customer_id,order_date,status,total_amount
0,1001,1,2024-06-01,delivered,4650.0
1,1002,2,2024-06-05,delivered,3050.0
2,1003,1,2024-06-10,shipped,1200.0
3,1004,3,2024-06-15,processing,2480.0
4,1005,4,2024-06-20,pending,850.0
5,1006,5,2024-06-25,delivered,4780.0
6,1007,2,2024-07-01,shipped,430.0


## Part 2: Migration with SHARDING
**Key Format:** `TableName:TupleID:Attribute` → Value

In [31]:
print('🚀 MIGRATING WITH SHARDING')
print('='*70)
keys_per_master = {0:[], 1:[], 2:[], 3:[]}

def set_sharded(key, value):
    mid = get_master_id(key)
    masters[mid].set(key, value)
    keys_per_master[mid].append(key)
    return mid, get_slot(key)

# Migrate Customers
print('\n👥 Migrating CUSTOMER...')
for row in cursor.execute('SELECT * FROM Customer').fetchall():
    for attr, val in [('first_name',row[1]),('last_name',row[2]),('email',row[3]),('phone',row[4]),('city',row[5]),('country',row[6])]:
        key = f'Customer:{row[0]}:{attr}'
        mid, slot = set_sharded(key, val)
        print(f'  SET {key:<35} → Master {mid} (slot {slot})')

🚀 MIGRATING WITH SHARDING

👥 Migrating CUSTOMER...
  SET Customer:1:first_name               → Master 1 (slot 5855)
  SET Customer:1:last_name                → Master 1 (slot 5355)
  SET Customer:1:email                    → Master 2 (slot 11839)
  SET Customer:1:phone                    → Master 3 (slot 14865)
  SET Customer:1:city                     → Master 1 (slot 6192)
  SET Customer:1:country                  → Master 2 (slot 10987)
  SET Customer:2:first_name               → Master 3 (slot 13376)
  SET Customer:2:last_name                → Master 0 (slot 2604)
  SET Customer:2:email                    → Master 2 (slot 11008)
  SET Customer:2:phone                    → Master 3 (slot 14034)
  SET Customer:2:city                     → Master 0 (slot 209)
  SET Customer:2:country                  → Master 1 (slot 7148)
  SET Customer:3:first_name               → Master 1 (slot 4513)
  SET Customer:3:last_name                → Master 3 (slot 16237)
  SET Customer:3:email           

In [32]:
# Migrate Products
print('📦 Migrating PRODUCT...')
for row in cursor.execute('SELECT * FROM Product').fetchall():
    for attr, val in [('product_name',row[1]),('category',row[2]),('price',str(row[3])),('stock_quantity',str(row[4]))]:
        key = f'Product:{row[0]}:{attr}'
        mid, slot = set_sharded(key, val)
        print(f'  SET {key:<35} → Master {mid} (slot {slot})')

📦 Migrating PRODUCT...
  SET Product:101:product_name            → Master 2 (slot 9352)
  SET Product:101:category                → Master 2 (slot 11589)
  SET Product:101:price                   → Master 3 (slot 14266)
  SET Product:101:stock_quantity          → Master 2 (slot 8521)
  SET Product:102:product_name            → Master 1 (slot 7721)
  SET Product:102:category                → Master 3 (slot 15974)
  SET Product:102:price                   → Master 3 (slot 13435)
  SET Product:102:stock_quantity          → Master 0 (slot 1834)
  SET Product:103:product_name            → Master 1 (slot 6090)
  SET Product:103:category                → Master 0 (slot 3975)
  SET Product:103:price                   → Master 3 (slot 12604)
  SET Product:103:stock_quantity          → Master 2 (slot 11531)
  SET Product:104:product_name            → Master 1 (slot 4459)
  SET Product:104:category                → Master 2 (slot 8360)
  SET Product:104:price                   → Master 2 (slot 11

In [33]:
# Migrate Orders
print('🛒 Migrating ORDER...')
for row in cursor.execute('SELECT * FROM Order_').fetchall():
    for attr, val in [('customer_id',str(row[1])),('order_date',row[2]),('status',row[3]),('total_amount',str(row[4]))]:
        key = f'Order:{row[0]}:{attr}'
        mid, slot = set_sharded(key, val)
        print(f'  SET {key:<35} → Master {mid} (slot {slot})')
print('\n✅ Migration complete!')

🛒 Migrating ORDER...
  SET Order:1001:customer_id              → Master 2 (slot 10160)
  SET Order:1001:order_date               → Master 3 (slot 12459)
  SET Order:1001:status                   → Master 3 (slot 16118)
  SET Order:1001:total_amount             → Master 0 (slot 3145)
  SET Order:1002:customer_id              → Master 3 (slot 12593)
  SET Order:1002:order_date               → Master 0 (slot 3596)
  SET Order:1002:status                   → Master 1 (slot 5079)
  SET Order:1002:total_amount             → Master 0 (slot 1514)
  SET Order:1003:customer_id              → Master 3 (slot 15026)
  SET Order:1003:order_date               → Master 2 (slot 11117)
  SET Order:1003:status                   → Master 2 (slot 10424)
  SET Order:1003:total_amount             → Master 3 (slot 16267)
  SET Order:1004:customer_id              → Master 0 (slot 1075)
  SET Order:1004:order_date               → Master 0 (slot 2254)
  SET Order:1004:status                   → Master 3 (slot 15

In [34]:
# =============================================================
# SECONDARY INDEX IMPLEMENTATION
# =============================================================

print("="*70)
print("📇 CREATING SECONDARY INDEXES")
print("="*70)

# Define which attributes to index
index_fields = {
    'Customer': ['city', 'country'],
    'Product': ['category'],
    'Order': ['status']
}

# ID ranges for each table
id_ranges = {
    'Customer': range(1, 6),
    'Product': range(101, 107),
    'Order': range(1001, 1008)
}

# Create secondary indexes
secondary_indexes = {}

for table, fields in index_fields.items():
    for field in fields:
        for record_id in id_ranges[table]:
            # Get the value from Redis
            key = f"{table}:{record_id}:{field}"
            mid = get_master_id(key)
            value = masters[mid].get(key)

            if value:
                # Create index key: idx:Table:field:value
                index_key = f"idx:{table}:{field}:{value}"

                # Add record_id to the index
                if index_key not in secondary_indexes:
                    secondary_indexes[index_key] = []
                secondary_indexes[index_key].append(record_id)

# Store indexes in Redis (on Master 0 for simplicity)
for index_key, record_ids in secondary_indexes.items():
    # Store as comma-separated string
    masters[0].set(index_key, ",".join(map(str, record_ids)))
    print(f"   {index_key} → {record_ids}")

print(f"\n✅ Created {len(secondary_indexes)} secondary indexes!")


# =============================================================
# QUERY FUNCTIONS USING SECONDARY INDEX
# =============================================================

def query_by_index(table, field, value):
    """
    Query using secondary index - O(1) lookup!

    Usage:
        query_by_index('Customer', 'city', 'Riyadh')
        query_by_index('Product', 'category', 'Electronics')
        query_by_index('Order', 'status', 'delivered')
    """
    print(f"\n{'='*70}")
    print(f"🔍 SECONDARY INDEX QUERY")
    print(f"   Table: {table} | Field: {field} | Value: {value}")
    print("="*70)

    # Build index key
    index_key = f"idx:{table}:{field}:{value}"

    print(f"\n1️⃣ Looking up index key: {index_key}")
    print(f"   Command: masters[0].get('{index_key}')")

    # Get record IDs from index (stored on Master 0)
    result = masters[0].get(index_key)

    if not result:
        print(f"\n❌ No records found where {field} = '{value}'")
        return

    record_ids = result.split(",")
    print(f"   Result: {record_ids}")
    print(f"\n2️⃣ Found {len(record_ids)} matching record(s)")

    # Define attributes for display
    display_attrs = {
        'Customer': ['first_name', 'last_name', 'email', 'city'],
        'Product': ['product_name', 'category', 'price'],
        'Order': ['customer_id', 'status', 'total_amount']
    }

    # Fetch and display each record
    print(f"\n3️⃣ Fetching records from Redis:")
    print("-"*70)

    for rid in record_ids:
        print(f"\n   📌 {table} {rid}:")
        for attr in display_attrs[table]:
            key = f"{table}:{rid}:{attr}"
            mid = get_master_id(key)
            val = masters[mid].get(key)
            if val:
                print(f"      {attr}: {val} [Master {mid}]")


def show_all_indexes():
    """Display all secondary indexes"""
    print(f"\n{'='*70}")
    print("📇 ALL SECONDARY INDEXES")
    print("="*70)

    # Get all index keys from Master 0
    index_keys = masters[0].keys("idx:*")

    print(f"\n{'Index Key':<40} {'Record IDs':<20}")
    print("-"*60)

    for idx_key in sorted(index_keys):
        value = masters[0].get(idx_key)
        print(f"{idx_key:<40} {value:<20}")

    print(f"\n📊 Total indexes: {len(index_keys)}")


def compare_query_methods(table, field, value):
    """
    Compare O(n) scan vs O(1) index lookup
    """
    import time

    print(f"\n{'='*70}")
    print(f"⚡ COMPARING QUERY METHODS")
    print(f"   Find all {table}s where {field} = '{value}'")
    print("="*70)

    # Method 1: Full Scan (O(n))
    print(f"\n📍 Method 1: FULL SCAN (No Index) - O(n)")
    print("-"*50)
    start = time.time()

    scan_results = []
    for rid in id_ranges[table]:
        key = f"{table}:{rid}:{field}"
        mid = get_master_id(key)
        val = masters[mid].get(key)
        if val == value:
            scan_results.append(rid)

    scan_time = time.time() - start
    print(f"   Scanned all {table} records")
    print(f"   Found: {scan_results}")
    print(f"   Time: {scan_time*1000:.3f} ms")

    # Method 2: Index Lookup (O(1))
    print(f"\n📍 Method 2: INDEX LOOKUP - O(1)")
    print("-"*50)
    start = time.time()

    index_key = f"idx:{table}:{field}:{value}"
    result = masters[0].get(index_key)
    index_results = result.split(",") if result else []

    index_time = time.time() - start
    print(f"   Single key lookup: {index_key}")
    print(f"   Found: {index_results}")
    print(f"   Time: {index_time*1000:.3f} ms")

    # Comparison
    print(f"\n📊 COMPARISON:")
    print(f"   Full Scan: {scan_time*1000:.3f} ms")
    print(f"   Index:     {index_time*1000:.3f} ms")
    if scan_time > 0:
        print(f"   Index is ~{scan_time/max(index_time, 0.0001):.1f}x faster")


# =============================================================
# USAGE INSTRUCTIONS
# =============================================================
print("\n" + "="*70)
print("✅ SECONDARY INDEX FUNCTIONS LOADED")
print("="*70)
print("\n📋 Available Commands:")
print("   query_by_index('Customer', 'city', 'Riyadh')")
print("   query_by_index('Product', 'category', 'Electronics')")
print("   query_by_index('Order', 'status', 'delivered')")
print("   show_all_indexes()")
print("   compare_query_methods('Customer', 'city', 'Riyadh')")

📇 CREATING SECONDARY INDEXES
   idx:Customer:city:Riyadh → [1, 4]
   idx:Customer:city:Jeddah → [2]
   idx:Customer:city:Dammam → [3]
   idx:Customer:city:Mecca → [5]
   idx:Customer:country:Saudi Arabia → [1, 2, 3, 4, 5]
   idx:Product:category:Electronics → [101, 102, 105, 106]
   idx:Product:category:Furniture → [103, 104]
   idx:Order:status:delivered → [1001, 1002, 1006]
   idx:Order:status:shipped → [1003, 1007]
   idx:Order:status:processing → [1004]
   idx:Order:status:pending → [1005]

✅ Created 11 secondary indexes!

✅ SECONDARY INDEX FUNCTIONS LOADED

📋 Available Commands:
   query_by_index('Customer', 'city', 'Riyadh')
   query_by_index('Product', 'category', 'Electronics')
   query_by_index('Order', 'status', 'delivered')
   show_all_indexes()
   compare_query_methods('Customer', 'city', 'Riyadh')


## Part 3: SHARDING DEMONSTRATION

In [35]:
print('🎯 SHARDING - Keys Distribution')
print('='*60)
for mid in range(4):
    print(f'\nMaster {mid} (Port {7000+mid}) - {len(keys_per_master[mid])} keys')
    for k in keys_per_master[mid][:3]:
        print(f'  {k} → "{masters[mid].get(k)}"')

🎯 SHARDING - Keys Distribution

Master 0 (Port 7000) - 16 keys
  Customer:2:last_name → "Hassan"
  Customer:2:city → "Jeddah"
  Customer:3:country → "Saudi Arabia"

Master 1 (Port 7001) - 14 keys
  Customer:1:first_name → "Ahmed"
  Customer:1:last_name → "Al-Rashid"
  Customer:1:city → "Riyadh"

Master 2 (Port 7002) - 28 keys
  Customer:1:email → "ahmed.rashid@email.com"
  Customer:1:country → "Saudi Arabia"
  Customer:2:email → "fatima.hassan@email.com"

Master 3 (Port 7003) - 24 keys
  Customer:1:phone → "+966501234567"
  Customer:2:first_name → "Fatima"
  Customer:2:phone → "+966502345678"


## Part 4: REPLICATION DEMONSTRATION
**Write to Master → Read from Replica**

In [36]:
print('🔄 REPLICATION TEST')
print('='*60)
time.sleep(1)

# Test existing data
key = 'Customer:1:first_name'
mid = get_master_id(key)
master_val = masters[mid].get(key)
replica_val = replicas[mid].get(key)

print(f'\nKey: {key}')
print(f'Master {mid} (Port {7000+mid}): "{master_val}"')
print(f'Replica {mid} (Port {7004+mid}): "{replica_val}"')
print('✅ Data matches!' if master_val == replica_val else '❌ Mismatch')

🔄 REPLICATION TEST

Key: Customer:1:first_name
Master 1 (Port 7001): "Ahmed"
Replica 1 (Port 7005): "Ahmed"
✅ Data matches!


In [37]:
# Test NEW write
print('\n📝 Writing NEW data to Master, Reading from Replica')
print('-'*60)
new_key = 'Customer:100:first_name'
new_val = 'TestCustomer'
mid = get_master_id(new_key)

print(f'WRITE to Master {mid}: SET {new_key} "{new_val}"')
masters[mid].set(new_key, new_val)
time.sleep(1)

print(f'READ from Replica {mid}: GET {new_key}')
replica_val = replicas[mid].get(new_key)
print(f'Result: "{replica_val}"')
print('\n✅ REPLICATION VERIFIED!' if replica_val == new_val else '❌ Failed')


📝 Writing NEW data to Master, Reading from Replica
------------------------------------------------------------
WRITE to Master 1: SET Customer:100:first_name "TestCustomer"
READ from Replica 1: GET Customer:100:first_name
Result: "TestCustomer"

✅ REPLICATION VERIFIED!


## Part 5: Key-Value Operations

In [38]:
print('📋 GET by Key - Customer 1')
print('='*60)
for attr in ['first_name','last_name','email','phone','city','country']:
    key = f'Customer:1:{attr}'
    mid = get_master_id(key)
    print(f'GET {key:<30} → "{masters[mid].get(key)}"')

📋 GET by Key - Customer 1
GET Customer:1:first_name          → "Ahmed"
GET Customer:1:last_name           → "Al-Rashid"
GET Customer:1:email               → "ahmed.rashid@email.com"
GET Customer:1:phone               → "+966501234567"
GET Customer:1:city                → "Riyadh"
GET Customer:1:country             → "Saudi Arabia"


In [39]:
print('📋 GET by Key - Product 101')
print('='*60)
for attr in ['product_name','category','price','stock_quantity']:
    key = f'Product:101:{attr}'
    mid = get_master_id(key)
    print(f'GET {key:<35} → "{masters[mid].get(key)}"')

📋 GET by Key - Product 101
GET Product:101:product_name            → "Laptop Pro 15"
GET Product:101:category                → "Electronics"
GET Product:101:price                   → "4500.0"
GET Product:101:stock_quantity          → "25"


## Part 6: Redis CLI Commands

In [40]:
print('🖥️ CLI - Read from Master 7001')
!redis-cli -p 7001 GET Customer:1:first_name
!redis-cli -p 7002 GET Customer:1:email

🖥️ CLI - Read from Master 7001
"Ahmed"
"ahmed.rashid@email.com"


In [41]:
print('🖥️ CLI - Read from Replica 7005 (Same data!)')
!redis-cli -p 7005 GET Customer:1:first_name
!redis-cli -p 7006 GET Customer:1:email

🖥️ CLI - Read from Replica 7005 (Same data!)
"Ahmed"
"ahmed.rashid@email.com"


In [42]:
print('🖥️ CLI - DBSIZE on each node')
for p in range(7000,7008):
    result = !redis-cli -p {p} DBSIZE
    t = 'Master' if p < 7004 else 'Replica'
    print(f'Port {p} ({t}): {result[0]}')

🖥️ CLI - DBSIZE on each node
Port 7000 (Master): (integer) 27
Port 7001 (Master): (integer) 15
Port 7002 (Master): (integer) 28
Port 7003 (Master): (integer) 24
Port 7004 (Replica): (integer) 27
Port 7005 (Replica): (integer) 15
Port 7006 (Replica): (integer) 28
Port 7007 (Replica): (integer) 24


In [20]:
# =============================================================
# INTERACTIVE QUERY SYSTEM
# =============================================================

def interactive_query():
    """
    User-friendly query interface for Redis data
    """
    print("="*70)
    print("🔍 INTERACTIVE REDIS QUERY SYSTEM")
    print("="*70)

    while True:
        # Main Menu
        print("\n📋 SELECT TABLE:")
        print("   1. Customer")
        print("   2. Product")
        print("   3. Order")
        print("   4. Exit")

        table_choice = input("\nEnter choice (1-4): ").strip()

        if table_choice == '4':
            print("\n👋 Goodbye!")
            break

        table_map = {'1': 'Customer', '2': 'Product', '3': 'Order'}
        if table_choice not in table_map:
            print("❌ Invalid choice. Try again.")
            continue

        table_name = table_map[table_choice]

        # Query Type Menu
        print(f"\n📋 QUERY TYPE FOR {table_name.upper()}:")
        print("   1. Show ALL records")
        print("   2. Show SPECIFIC record by ID")
        print("   3. Show SPECIFIC attribute for ALL records")
        print("   4. Back to main menu")

        query_choice = input("\nEnter choice (1-4): ").strip()

        if query_choice == '4':
            continue

        if query_choice == '1':
            show_all_records(table_name)
        elif query_choice == '2':
            show_specific_record(table_name)
        elif query_choice == '3':
            show_attribute_for_all(table_name)
        else:
            print("❌ Invalid choice. Try again.")


def show_all_records(table_name):
    """Show all records from a table"""
    print(f"\n{'='*70}")
    print(f"📊 ALL {table_name.upper()} RECORDS")
    print("="*70)

    # Define attributes for each table
    attributes = {
        'Customer': ['first_name', 'last_name', 'email', 'phone', 'city', 'country'],
        'Product': ['product_name', 'category', 'price', 'stock_quantity'],
        'Order': ['customer_id', 'order_date', 'status', 'total_amount']
    }

    # Define ID ranges for each table
    id_ranges = {
        'Customer': range(1, 6),      # 1-5
        'Product': range(101, 107),   # 101-106
        'Order': range(1001, 1008)    # 1001-1007
    }

    attrs = attributes[table_name]
    ids = id_ranges[table_name]

    for record_id in ids:
        print(f"\n🔹 {table_name} ID: {record_id}")
        print("-"*60)

        for attr in attrs:
            key = f"{table_name}:{record_id}:{attr}"
            slot = get_slot(key)
            mid = get_master_id(key)
            value = masters[mid].get(key)

            if value:
                print(f"   {attr:<18} = {value:<25} [Master {mid} | Slot {slot}]")


def show_specific_record(table_name):
    """Show a specific record by ID"""

    # Define ID hints
    id_hints = {
        'Customer': '1-5',
        'Product': '101-106',
        'Order': '1001-1007'
    }

    # Define attributes for each table
    attributes = {
        'Customer': ['first_name', 'last_name', 'email', 'phone', 'city', 'country'],
        'Product': ['product_name', 'category', 'price', 'stock_quantity'],
        'Order': ['customer_id', 'order_date', 'status', 'total_amount']
    }

    record_id = input(f"\nEnter {table_name} ID ({id_hints[table_name]}): ").strip()

    print(f"\n{'='*70}")
    print(f"📊 {table_name.upper()} RECORD: {record_id}")
    print("="*70)

    found = False
    for attr in attributes[table_name]:
        key = f"{table_name}:{record_id}:{attr}"
        slot = get_slot(key)
        mid = get_master_id(key)
        value = masters[mid].get(key)

        if value:
            found = True
            print(f"   {attr:<18} = {value:<25} [Master {mid} | Port {7000+mid} | Slot {slot}]")

    if not found:
        print(f"   ❌ No record found with ID {record_id}")


def show_attribute_for_all(table_name):
    """Show a specific attribute for all records"""

    # Define attributes for each table
    attributes = {
        'Customer': ['first_name', 'last_name', 'email', 'phone', 'city', 'country'],
        'Product': ['product_name', 'category', 'price', 'stock_quantity'],
        'Order': ['customer_id', 'order_date', 'status', 'total_amount']
    }

    # Define ID ranges for each table
    id_ranges = {
        'Customer': range(1, 6),
        'Product': range(101, 107),
        'Order': range(1001, 1008)
    }

    attrs = attributes[table_name]

    print(f"\n📋 AVAILABLE ATTRIBUTES FOR {table_name.upper()}:")
    for i, attr in enumerate(attrs, 1):
        print(f"   {i}. {attr}")

    attr_choice = input("\nEnter attribute number: ").strip()

    try:
        attr_idx = int(attr_choice) - 1
        if 0 <= attr_idx < len(attrs):
            selected_attr = attrs[attr_idx]
        else:
            print("❌ Invalid choice")
            return
    except:
        print("❌ Invalid input")
        return

    print(f"\n{'='*70}")
    print(f"📊 ALL {selected_attr.upper()} VALUES FROM {table_name.upper()}")
    print("="*70)
    print(f"\n{'ID':<10} {'Value':<30} {'Master':<10} {'Port':<10} {'Slot':<10}")
    print("-"*70)

    for record_id in id_ranges[table_name]:
        key = f"{table_name}:{record_id}:{selected_attr}"
        slot = get_slot(key)
        mid = get_master_id(key)
        value = masters[mid].get(key)

        if value:
            print(f"{record_id:<10} {value:<30} {mid:<10} {7000+mid:<10} {slot:<10}")


# =============================================================
# QUICK QUERY FUNCTIONS (Alternative - Non-Interactive)
# =============================================================

def query_customer(customer_id=None):
    """
    Query customer data
    Usage: query_customer()      - shows all customers
           query_customer(1)     - shows customer 1
    """
    if customer_id:
        show_record('Customer', customer_id, ['first_name', 'last_name', 'email', 'phone', 'city', 'country'])
    else:
        for cid in range(1, 6):
            show_record('Customer', cid, ['first_name', 'last_name', 'email', 'phone', 'city', 'country'])
            print()


def query_product(product_id=None):
    """
    Query product data
    Usage: query_product()       - shows all products
           query_product(101)    - shows product 101
    """
    if product_id:
        show_record('Product', product_id, ['product_name', 'category', 'price', 'stock_quantity'])
    else:
        for pid in range(101, 107):
            show_record('Product', pid, ['product_name', 'category', 'price', 'stock_quantity'])
            print()


def query_order(order_id=None):
    """
    Query order data
    Usage: query_order()         - shows all orders
           query_order(1001)     - shows order 1001
    """
    if order_id:
        show_record('Order', order_id, ['customer_id', 'order_date', 'status', 'total_amount'])
    else:
        for oid in range(1001, 1008):
            show_record('Order', oid, ['customer_id', 'order_date', 'status', 'total_amount'])
            print()


def show_record(table_name, record_id, attributes):
    """Helper function to display a record with node info"""
    print(f"🔹 {table_name} {record_id}:")
    for attr in attributes:
        key = f"{table_name}:{record_id}:{attr}"
        slot = get_slot(key)
        mid = get_master_id(key)
        value = masters[mid].get(key)
        if value:
            print(f"   {attr:<18} = {value:<25} [Master {mid} | Slot {slot}]")


# =============================================================
# SEARCH BY VALUE FUNCTION
# =============================================================

def search_by_value(search_term):
    """
    Search for a value across all tables
    Usage: search_by_value("Ahmed")
           search_by_value("delivered")
           search_by_value("Riyadh")
    """
    print(f"\n{'='*70}")
    print(f"🔍 SEARCHING FOR: '{search_term}'")
    print("="*70)

    found_count = 0

    # Search all tables
    tables = {
        'Customer': (range(1, 6), ['first_name', 'last_name', 'email', 'phone', 'city', 'country']),
        'Product': (range(101, 107), ['product_name', 'category', 'price', 'stock_quantity']),
        'Order': (range(1001, 1008), ['customer_id', 'order_date', 'status', 'total_amount'])
    }

    for table_name, (id_range, attributes) in tables.items():
        for record_id in id_range:
            for attr in attributes:
                key = f"{table_name}:{record_id}:{attr}"
                slot = get_slot(key)
                mid = get_master_id(key)
                value = masters[mid].get(key)

                if value and search_term.lower() in value.lower():
                    found_count += 1
                    print(f"\n✅ FOUND in {table_name} (ID: {record_id})")
                    print(f"   Key:    {key}")
                    print(f"   Value:  {value}")
                    print(f"   Master: {mid} (Port {7000+mid})")
                    print(f"   Slot:   {slot}")

    if found_count == 0:
        print(f"\n❌ No results found for '{search_term}'")
    else:
        print(f"\n📊 Total results: {found_count}")


# =============================================================
# RUN INTERACTIVE QUERY
# =============================================================
print("✅ Interactive Query System Loaded!")
print("\nAvailable commands:")
print("   interactive_query()     - Start interactive menu")
print("   query_customer()        - Show all customers")
print("   query_customer(1)       - Show customer 1")
print("   query_product()         - Show all products")
print("   query_product(101)      - Show product 101")
print("   query_order()           - Show all orders")
print("   query_order(1001)       - Show order 1001")
print("   search_by_value('Ahmed') - Search for a value")

✅ Interactive Query System Loaded!

Available commands:
   interactive_query()     - Start interactive menu
   query_customer()        - Show all customers
   query_customer(1)       - Show customer 1
   query_product()         - Show all products
   query_product(101)      - Show product 101
   query_order()           - Show all orders
   query_order(1001)       - Show order 1001
   search_by_value('Ahmed') - Search for a value


In [21]:
query_customer()        # Show ALL customers
query_customer(1)       # Show customer 1 only

query_product()         # Show ALL products
query_product(101)      # Show product 101 only

query_order()           # Show ALL orders
query_order(1001)       # Show order 1001 only

🔹 Customer 1:
   first_name         = Ahmed                     [Master 1 | Slot 5855]
   last_name          = Al-Rashid                 [Master 1 | Slot 5355]
   email              = ahmed.rashid@email.com    [Master 2 | Slot 11839]
   phone              = +966501234567             [Master 3 | Slot 14865]
   city               = Riyadh                    [Master 1 | Slot 6192]
   country            = Saudi Arabia              [Master 2 | Slot 10987]

🔹 Customer 2:
   first_name         = Fatima                    [Master 3 | Slot 13376]
   last_name          = Hassan                    [Master 0 | Slot 2604]
   email              = fatima.hassan@email.com   [Master 2 | Slot 11008]
   phone              = +966502345678             [Master 3 | Slot 14034]
   city               = Jeddah                    [Master 0 | Slot 209]
   country            = Saudi Arabia              [Master 1 | Slot 7148]

🔹 Customer 3:
   first_name         = Mohammed                  [Master 1 | Slot 4513]
  

In [ ]:
search_by_value("Ahmed")      # Find all records containing "Ahmed"
search_by_value("delivered")  # Find all delivered orders
search_by_value("Riyadh")     # Find all records in Riyadh

In [22]:
interactive_query()

🔍 INTERACTIVE REDIS QUERY SYSTEM

📋 SELECT TABLE:
   1. Customer
   2. Product
   3. Order
   4. Exit

Enter choice (1-4): 1

📋 QUERY TYPE FOR CUSTOMER:
   1. Show ALL records
   2. Show SPECIFIC record by ID
   3. Show SPECIFIC attribute for ALL records
   4. Back to main menu

Enter choice (1-4): 1

📊 ALL CUSTOMER RECORDS

🔹 Customer ID: 1
------------------------------------------------------------
   first_name         = Ahmed                     [Master 1 | Slot 5855]
   last_name          = Al-Rashid                 [Master 1 | Slot 5355]
   email              = ahmed.rashid@email.com    [Master 2 | Slot 11839]
   phone              = +966501234567             [Master 3 | Slot 14865]
   city               = Riyadh                    [Master 1 | Slot 6192]
   country            = Saudi Arabia              [Master 2 | Slot 10987]

🔹 Customer ID: 2
------------------------------------------------------------
   first_name         = Fatima                    [Master 3 | Slot 13376]
 

KeyboardInterrupt: Interrupted by user

## Cluster Architecture
```
MASTER 0 (7000) ←→ REPLICA 0 (7004)  Slots 0-4095
MASTER 1 (7001) ←→ REPLICA 1 (7005)  Slots 4096-8191
MASTER 2 (7002) ←→ REPLICA 2 (7006)  Slots 8192-12287
MASTER 3 (7003) ←→ REPLICA 3 (7007)  Slots 12288-16383
```

## ✅ Conclusion
| Topic | Status |
|-------|--------|
| Mapping | TableName:TupleID:Attribute → Value |
| Sharding | ✅ Keys distributed across 4 masters |
| Replication | ✅ Write Master, Read Replica works |
| Operations | SET, GET, KEYS demonstrated |